In [ ]:
import sys

sys.path.append('../..')

import pandas as pd

from src.tools import *
from src.database import Database


In [ ]:
# lista delle colonne del database delle transazioni
columns = Database.txs_column_list
columns


importo le transazioni di `./data/transazioni.xsls` in un Dataframe e le adatto allo standard.
Verifico che il formato sia giusto attraverso la funzione `src.arrange`

In [ ]:
timezone = 'UTC'

def all(): 
    df = pd.read_excel('data/transazioni.xlsx').sort_values(by='id')
    agg = ['deposito', 'Acquisto valuta', 'dividend_from_shares', 'sold_shares' ]
    sub = ['prelievo', 'Vendita valuta', 'bought_shares','return_lent_currency']
    comm = ['pagamento commissione']
    
    txs = pd.DataFrame()
    txs['id']=df['id']
    txs['timestamp'] = (df['date_operation'].apply(str).apply(lambda x : x[:10]+' ' ) \
                    + df['time_operation'].apply(str)\
                        ).apply(lambda x :timeFormat(x, input_timezone=timezone))
    
    for i, r in df.iterrows():
        tipo = r['Tipo']
        if tipo in agg:
            txs.loc[i, 'in'] = r['deposito']
            txs.loc[i, 'in_iso'] = r['valuta']
        elif tipo in sub:
            txs.loc[i, 'out'] =  r['deposito']
            txs.loc[i, 'out_iso'] = r['valuta']
        elif tipo in comm:
            txs.loc[i, 'fee'] = r['deposito']
            txs.loc[i, 'fee_iso'] = r['valuta']

        if (tipo == 'deposito' or tipo == 'prelievo'):
            txs.loc[i, 'type'] = 'ext'
            txs.loc[i, 'info'] = r['bt_tx_address']
        elif (tipo == 'Acquisto valuta' or tipo == 'Vendita valuta'):
            txs.loc[i, 'type'] = 'trade'
        else:
            txs.loc[i, 'type'] = 'other'
            txs.loc[i, 'info'] = r['Tipo'] 
        
    txs.sort_values(by='timestamp', inplace=True)



    return arrange(txs)


txs = all()

Compattiamo le transazioni. Negli excel di `therocktrading` le quantità di valuta ceduta e acquistata in un trade sono registrate separatamente. Selezioniamo le transazioni che distano meno di un secondo e assegniamo loro lo stesso timestamp.

In [ ]:
def compact(df):
    txs = df.copy()
    t0 = 0
    for i, r in txs.iterrows():
        t1 = r['timestamp']
        if (t1-t0)<=1000:
            txs.loc[i, 'timestamp']=t0
            t1 = t0
        t0 = t1
    #verifica
    t0 = 0
    for i, r in txs.iterrows():
        t1 = r['timestamp']
        if (t1-t0)==1000:
            raise Exception('???', t0, t1)
        t0 = t1

    return txs

c = compact(txs.copy())

raggruppiamo le transazioni con uguale timestamp:

In [ ]:
grouped = c[c['type']=='trade'].groupby('timestamp')

In [ ]:
df = pd.DataFrame(columns=columns).set_index('id')
for t, group in grouped:
    #print('#########################@@@ group @@@', t, len(group))
    if len(group)>0:
        subgroup_in = group.groupby('in_iso') 
        subgroup_out  = group.groupby('out_iso')
        lenin = len(subgroup_in)
        lenout = len(subgroup_out)
        assert lenin == lenout
        if lenin == 0 or lenin > 2:
            raise Exception('esagerato', lenin)
        else:
            subin = subgroup_in.groups
            subout = subgroup_out.groups
            # print(subin, set(subin) )
            # print(subout, set(subout))

            if lenin == 2:
                assert set(subin) == set(subout)

            aggr_in = []
            for iso in subin:
                aggr_in.append({
                    "index":list(subin[iso]),
                    "in_iso":iso,
                    "in":subgroup_in.get_group(iso)['in'].sum()
                })
            #print('aggr_in', aggr_in)  

            aggr_out = []
            for iso in subout:
                aggr_out.append({
                    "index":list(subout[iso]),
                    "out_iso":iso,
                    "out":subgroup_out.get_group(iso)['out'].sum()
                })
            #print('aggr_out', aggr_out)

            def join(i, j):
                df.loc['_'.join(map(str,aggr_in[i]['index']+aggr_out[j]['index']))] = {
                    'timestamp':t,
                    'type':'trade',
                    'info':'aggregated',
                    'in_iso': aggr_in[i]['in_iso'],
                    'in': aggr_in[i]['in'],
                    'out_iso': aggr_out[j]['out_iso'],
                    'out': aggr_out[j]['out']
                }

            if len(aggr_in) == 1:
                join(0,0)
            elif len(aggr_in) == 2:
                if aggr_in[0]['in_iso'] == aggr_out[0]['out_iso']:
                    join(0,1)
                    join(1,0)
                else:
                    join(0,0)
                    join(1,1)
            

In [ ]:
for i, r in txs.iterrows():
    if r['type'] != 'trade':
        df.loc[i] = r

In [ ]:
df.sort_values(by='timestamp', inplace=True)
df.reset_index(inplace=True)
df = arrange(df)

In [ ]:
balances = makeBalances(df)

In [ ]:
m = maxi(df, balances, True)
db = Database('db.db')
name = 'therocktrading'
db.insert_balances(name, balances)
db.insert_txs(name, df)